In [1]:
from google.colab import files
uploaded = files.upload()

Saving requirements.txt to requirements.txt


In [2]:
uploaded

{'requirements.txt': b'streamlit\ngoogle-generativeai\npython-dotenv\nlangchain\nPyPDF2\nchromadb\nfaiss-cpu'}

In [10]:
uploaded2 = files.upload()

Saving vision.py to vision.py


In [11]:
!pip install -r requirements.txt

In [12]:
!pip install pyngrok

In [17]:
from dotenv import load_dotenv
import streamlit as st
import os
import pathlib
import textwrap
from PIL import Image
import google.generativeai as genai
from pyngrok import ngrok  # Add this import for tunneling


# Load environment variables
load_dotenv()

os.environ.get('NGROK_AUTH_TOKEN')
# Configure the Google Gemini API
genai.configure(api_key= os.environ.get('GOOGLE_API_KEY'))



# Function to load OpenAI model and get responses
def get_gemini_response(input, image, prompt):
    model = genai.GenerativeModel('gemini-pro-vision')
    response = model.generate_content([input, image[0], prompt])
    return response.text

def input_image_setup(uploaded_file):
    # Check if a file has been uploaded
    if uploaded_file is not None:
        # Read the file into bytes
        bytes_data = uploaded_file.getvalue()

        image_parts = [
            {
                "mime_type": uploaded_file.type,  # Get the mime type of the uploaded file
                "data": bytes_data
            }
        ]
        return image_parts
    else:
        raise FileNotFoundError("No file uploaded")

# Initialize our Streamlit app
st.set_page_config(page_title="Gemini Image Demo")

st.header("Gemini Application")
input = st.text_input("Input Prompt: ", key="input")
uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])
image = ""
if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption="Uploaded Image.", use_column_width=True)

submit = st.button("Tell me about the image")

input_prompt = """
               You are an expert in understanding invoices.
               You will receive input images as invoices &
               you will have to answer questions based on the input image
               """

# If ask button is clicked
if submit:
    image_data = input_image_setup(uploaded_file)
    response = get_gemini_response(input_prompt, image_data, input)
    st.subheader("The Response is")
    st.write(response)

# --- New section for Google Colab ---
# Explicitly specify the tunnel protocol and options
tunnel = ngrok.connect(8000, proto='http', bind_tls=True)  # Use 'http' and bind_tls for Streamlit
public_url = tunnel.public_url
print(f"Streamlit app is running at: {public_url}")

# Run the Streamlit app (ensure app.py exists and is in the same directory)
!streamlit run vision.py --server.port 8000

Streamlit app is running at: https://0ab9-34-138-166-62.ngrok-free.app



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8000
  Network URL: http://172.28.0.12:8000
  External URL: http://34.138.166.62:8000

2024-08-26 04:16:11.223 Uncaught app exception
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/streamlit/runtime/scriptrunner/exec_code.py", line 85, in exec_func_with_error_handling
    result = func()
  File "/usr/local/lib/python3.10/dist-packages/streamlit/runtime/scriptrunner/script_runner.py", line 576, in code_to_exec
    exec(code, module.__dict__)
  File "/content/vision.py", line 64, in <module>
    response = get_gemini_response(input_prompt, image_data, input)
  File "/content/vision.py", line 23, in get_gemini_response
    response = model.generate_content([input, image[0], prompt])
  File "/usr/local/lib/python3.10/dist-packages/google/generativeai/generative_models.py", line 317, in generate_co